## Target- urgent withdrawn by nurse(within 24hr)
* shift_applications status == 'cancelled' + shift_application prevStatus == 'confirmed'


### IDs -> only look at shifts application, leave each shifts after
* shift_applications id
* shift_applications user_id
* shift_applications shift_id


### Numeric Field
* shifts rate
* shifts net_pay: rate - nursedash profit
* shift_applications distance
* shift break time
* user withdrawn times: 
    Problem: 
        1. We don't know in reality that you will withdraw your previous one.
        2. cannot be used to predict new user
* user applied times:
* prev SA/CW rate


## Time
### shift, shift app time
* S_Create (shift created) => SA_Create (application created) => CWTime (comfim withdrawn) => Start_Time (shift start)
* S_Create: shift create time
* SA_Create: shift application create time
* CWTime: shift withdrawn time, whole confirmed withdrawn population (include within 24hr and > 24hr)
* Start_Time: shift start time
##### calculated field, graph in hours
* S_Create2SA_Create: shift application create time - shift create time
* S_Create2Start_Time: shift start time - shift create time
        1. has negative values. delete maybe?
* SA_Create2Start_Time: shift start time - shift application create time
        1. has negative values. delete maybe?

### User
* user approvedAt:
* user createdAt:
#### Calculated field
* U_Create2U_approved: User approve time - User create time
* U_Create2now: now - User create time, in months(seconds/2629746)
* U_Approve2now: now - User approve time


### Categorical
* shifts role: position name + type
* facilities name = facilities short_name
* facilities areaId = facilities areaName
* withdrawnInfo_key, withdrawnInfo_value
* facilities segmentName: Senior Living = 1, Healthcare = 0
* Users enableNotifications:
* Users emailNotifications:
* Users appNotifications:
* Users allowedNotifications:


### ?
* facilities allowedQualification
* facilities createdAt
* facilities rates
* nurse shift withdrawn by admin, when policy start, how many withdrawn by admin
* shifts = unit
* role
* users updateAt ?
* users relationToFacility

# Connect to DB, Fetch data

In [81]:
import pandas as pd
# specify connection to database
import psycopg2
connection = psycopg2.connect(
    host="nursedash-prod.cuzi2kducsnv.us-east-1.rds.amazonaws.com",
    database="nursedash",
    user="external_analyst",
    password="uDps8APganhSLc3K2xe7NtMPq")

### <font color = green> all time to chicago time, No withdrawn info

In [82]:
df = pd.read_sql_query("""

SELECT  sa.id, sa.user_id, sa.shift_id, f.id AS facility_id, sa."withdrawnInfo" -> 'initiator' as withdrawnInfo_value,
sa."status", sa."prevStatus", sa."distance", s."facility_id", "s"."description" AS "shift_description",
"s"."assigned_nurse_id", s."net_pay", "s"."unit" AS "s_unit",s."type",
"s"."qualifications" AS "s_qualifications", "s"."breakTime" AS "s_breakTime", sa."withdrawnInfo",
"f"."name" AS "f_name","f"."short_name" AS "f_short_name", f."segmentName", f."areaName",
timezone('America/Chicago', s."createdAt") as s_create,
timezone('America/Chicago', sa."createdAt") as sa_create,
timezone('America/Chicago', u."approvedAt") as u_approve,
timezone('America/Chicago', u."createdAt") as u_create,
timezone('America/Chicago', sa."statusUpdatedAt") as sa_statusUpdate,
timezone('America/Chicago', timezone('UTC', s.start_time)) AS "Start_Time" 
FROM shifts s
INNER JOIN shift_applications sa ON s.id = sa.shift_id
INNER JOIN facilities f ON s.facility_id = f.id
INNER JOIN users u ON sa.user_id = u.id

""", con = connection)

In [83]:
df.columns

Index(['id', 'user_id', 'shift_id', 'facility_id', 'withdrawninfo_value',
       'status', 'prevStatus', 'distance', 'facility_id', 'shift_description',
       'assigned_nurse_id', 'net_pay', 's_unit', 'type', 's_qualifications',
       's_breakTime', 'withdrawnInfo', 'f_name', 'f_short_name', 'segmentName',
       'areaName', 's_create', 'sa_create', 'u_approve', 'u_create',
       'sa_statusupdate', 'Start_Time'],
      dtype='object')

### <font color = green> these features was done in sql before

### U_create2U_approve, in hours

In [84]:
from datetime import datetime
df['U_create2U_approve'] = df.apply(lambda row: (row['u_approve'] - row['u_create']).total_seconds()/3600, 
                                    axis = 1)

### U_create2now, in month

In [85]:
df['U_create2now'] = df.apply(lambda row: (datetime.now() - row['u_create']).total_seconds()/2629746, 
                              axis = 1)

### U_approve2now, in month

In [86]:
df['U_approve2now'] = df.apply(lambda row: (datetime.now() - row['u_approve']).total_seconds()/2629746, 
                               axis = 1)

### S_create2SA_Create, in hours

In [87]:
df['S_create2SA_Create'] = df.apply(lambda row: (row['sa_create'] - row['s_create']).total_seconds()/3600, 
                                    axis = 1)

### SA_Create2Start_Time

In [88]:
# some start time is at year 0019 
df['SA_Create2Start_Time'] = df.apply(lambda row: (row['Start_Time'] - row['sa_create']).total_seconds()/3600 if str(row['Start_Time'])[:2] != '00' else 9999, 
                                         axis = 1)

### S_Create2Start_Time

In [89]:
df['S_Create2Start_Time'] = df.apply(lambda row: (row['Start_Time'] - row['s_create']).total_seconds()/3600 if str(row['Start_Time'])[:2] != '00' else 9999, 
                                         axis = 1)

### CW_Time2Start_Time

In [90]:
# CW_Time2Start_Time
df['CW_Time2Start_Time'] = df.apply(lambda row: (row['Start_Time'] - row['sa_statusupdate']).total_seconds()/3600 if str(row['Start_Time'])[:2] != '00' else 9999, 
                                         axis = 1)

# Data Cleaning

In [91]:
df.columns

Index(['id', 'user_id', 'shift_id', 'facility_id', 'withdrawninfo_value',
       'status', 'prevStatus', 'distance', 'facility_id', 'shift_description',
       'assigned_nurse_id', 'net_pay', 's_unit', 'type', 's_qualifications',
       's_breakTime', 'withdrawnInfo', 'f_name', 'f_short_name', 'segmentName',
       'areaName', 's_create', 'sa_create', 'u_approve', 'u_create',
       'sa_statusupdate', 'Start_Time', 'U_create2U_approve', 'U_create2now',
       'U_approve2now', 'S_create2SA_Create', 'SA_Create2Start_Time',
       'S_Create2Start_Time', 'CW_Time2Start_Time'],
      dtype='object')

In [92]:
# There are some negative numbers in these column. Target all = 0, drop for now
df = df[(df["S_Create2Start_Time"]>0) & (df["SA_Create2Start_Time"]>0)]

# sort by time that shift was created
df = df.sort_values(by = 'sa_create').reset_index(drop = True)

In [93]:
# what are prevstatus column like when status = confirmed
df[df['status'] == 'confirmed']['prevStatus'].value_counts()

selected       47897
unavailable       17
applied            4
cancelled          2
Name: prevStatus, dtype: int64

In [94]:
# what are status column like when prevstatus = confirmed
df[df['prevStatus'] == 'confirmed']['status'].value_counts()

withdrawn    16664
cancelled     8232
applied          2
Name: status, dtype: int64

# Feature Engineering

### Target: urgent withdrawn as 1

In [95]:
def CW_by_nurse(row):
    if row['status']=='withdrawn' and row['prevStatus'] == 'confirmed':
        if row['withdrawninfo_value'] == 'nurse':
            if row['CW_Time2Start_Time'] <= 24:
                return 1
            else:
                return 0
        else:
            return 0
    else:
        return 0
    
df['target'] = df.apply (lambda row: CW_by_nurse(row), axis=1)

In [96]:
df['target'].value_counts()

0    186246
1      5808
Name: target, dtype: int64

### find extreme UCW rate facility from Top 30

In the TOP 30 facilities, Ohio Living Breckenridge Village,The Weils and Park Manor of South Belt has the highest UCW rate.

Whileas, The Hallmark and Village of Meyerland has the lowest UCW rate

In [97]:
df['f_highrate'] = df.apply(lambda row:1 if row['f_name'] in ['Ohio Living Breckenridge Village',
                                                              'The Weils', 
                                                              'Park Manor of South Belt'] else 0, axis = 1)
df['f_lowrate'] = df.apply(lambda row:1 if row['f_name'] in ['The Hallmark', 
                                                              'Village of Meyerland'] else 0, axis = 1)

###  Count previsous shift application

In [98]:
# track by using dictionary
count_prev_SA = []
test = df[['user_id', 'sa_create']]

# create SA dictionary, set all value = 0 -> dramatically reduce computational cost
uid_library = list(pd.unique(test['user_id']))
sa_dict = {} 
for uid in uid_library:
    sa_dict.update({uid: 0}) 
    
for i, v in enumerate(test['user_id']):
    sa_dict[v] += 1
    count_prev_SA.append(sa_dict[v]-1)
 
df['count_prev_SA'] = count_prev_SA

###  Count previsous urgent withdrawns

In [99]:
# track by using dictionary
count_prev_CW = []
test = df[['user_id', 'count_prev_SA', 'sa_create', 'target']]

# create cw dictionary, set all value = 0 -> dramatically reduce computational cost
uid_library = list(pd.unique(test['user_id']))
cw_dict = {} 
for uid in uid_library:
    cw_dict.update({uid: 0}) 
    
# fill dictionary and fill cw count
for i, v in enumerate(test['user_id']):
    if test['target'][i] == 1:
        cw_dict[v] += 1
        count_prev_CW.append(cw_dict[v]-1)
    else:
        count_prev_CW.append(cw_dict[v])
        
df['count_prev_CW'] = count_prev_CW

### Count previsous urgent withdrawns/Count previsous shift applications

In [100]:
df['prev_CW/SA_rate'] = df['count_prev_CW']/df['count_prev_SA']

# fill nan with 0, happend bc 0/0, meaning rate = 0
df['prev_CW/SA_rate'] = df['prev_CW/SA_rate'].fillna(0)

### Previous withdrawn times previous apply

In [101]:
df['prev_CW x SA_rate'] = df['count_prev_CW']*df['count_prev_SA']

### 2 Type dummy: RN, LVN + LPN , Rest


In [102]:
df['type'].value_counts()

CNA             102377
STNA             39421
LVN              18510
RN               14539
CMA/Med-Tech      7955
LPN               6440
Tech              2811
rn                   1
Name: type, dtype: int64

In [103]:
df['type_RN'] = df.apply(lambda row:1 if row['type'] == 'RN' else 0, axis = 1)
df['type_LVN+LPN'] = df.apply(lambda row: 1 if row['type'] == 'LVN' or row['type'] == 'LPN' else 0, axis = 1)


In [104]:
df['type_STNA'] = df.apply(lambda row:1 if row['type'] == 'STNA' else 0, axis = 1)

In [105]:
df['type_CNA'] = df.apply(lambda row:1 if row['type'] == 'CNA' else 0, axis = 1)

### 1 SegementName dummy: Senior Living, Healthcare

In [106]:
df['segmentName'].value_counts()

Senior Living    167445
Healthcare        24609
Name: segmentName, dtype: int64

In [107]:
def create_segmentName_dummy(row):
    if row['segmentName']=='Senior Living':
        return 1
    else:
        return 0
    
df['segmentName_d'] = df.apply(lambda row: create_segmentName_dummy(row), axis=1)

### 5 Facility Area Name Dummy

In [108]:
df['areaName'].value_counts()

Houston           108273
Northeast Ohio     45986
DFW                26350
Austin              6334
San Antonio         4603
Cincinnati           508
Name: areaName, dtype: int64

In [109]:
# Houston
df['areaName_houston'] = df.apply(lambda row: 1 if row['areaName'] == 'Houston' else 0, axis = 1)

# Northeast Ohio
df['areaName_no'] = df.apply(lambda row: 1 if row['areaName'] == 'Northeast Ohio' else 0, axis = 1)

# DFW
df['areaName_dfw'] = df.apply(lambda row: 1 if row['areaName'] == 'DFW' else 0, axis = 1)

# Austin
df['areaName_austin'] = df.apply(lambda row: 1 if row['areaName'] == 'Austin' else 0, axis = 1)

# San Antonio
df['areaName_san'] = df.apply(lambda row: 1 if row['areaName'] == 'San Antonio' else 0, axis = 1)

### keep only prevstatus or status = confirmed

In [110]:
df = df[df.apply(lambda row: (row['prevStatus'] == 'confirmed') or (row['status'] == 'confirmed'), axis = 1)].reset_index(drop = True)

In [111]:
# how many left
df.shape

(77063, 51)

### How many previsous urgent withdrawns in a month

In [112]:
import datetime
from datetime import datetime
from dateutil import relativedelta

# new column, convert start time to string
df['Str_Start_Time'] = df['Start_Time'].apply(lambda row: str(row))
# get only year, month, day in starttime
df['start_date'] = [w.split()[0] for w in list(df['Str_Start_Time'])]

# track by using dictionary, sort temp dataframe by userid then startdate
CW_in_a_month = []
df = df.sort_values(by=['user_id','start_date'], ascending=True).reset_index(drop = True)
test = df[['user_id', 'start_date', 'sa_create', 'target']].reset_index(drop = True)


for i, v in enumerate(test['user_id']):
    a = 0
    CW = 0
    delta = relativedelta.relativedelta(pd.to_datetime(test['start_date'][i-a]),
                                        pd.to_datetime(test['start_date'][i]))
    
    while delta.months > -1 and test['user_id'][i-a] == v: 
        if test['target'][i-a] == 1 and test['user_id'][i-a] == v and a >= 1:
            CW = 1
            break
    
        elif i-a <= 1:
            break

        elif test['user_id'][i-a] != v:
            break
            
        else:
            a += 1
            delta = relativedelta.relativedelta(pd.to_datetime(test['start_date'][i-a]),
                                                pd.to_datetime(test['start_date'][i]))
    CW_in_a_month.append(CW)

df['CW_in_a_month'] = CW_in_a_month

In [113]:
test1 = df[['user_id', 'start_date', 'CW_Time2Start_Time', 'target','status','prevStatus','withdrawninfo_value']]

In [114]:
reliability_score = []


for i, v in enumerate(test1['user_id']):
    CW = 40
    
    if i==0:
        reliability_score.append(50)
    else:
        if test1['user_id'][i] != test1['user_id'][i-1]:
            reliability_score.append(50)
        else:
            if test1['status'][i]=='withdrawn' and test1['prevStatus'][i] == 'confirmed' and test1['withdrawninfo_value'][i] == 'nurse':
                time = test1['CW_Time2Start_Time'][i]
                if time < 5:
                    score = reliability_score[-1] - 30
                    reliability_score.append(score)
                elif time >= 5 and time < 24:
                    score = reliability_score[-1] - 15
                    reliability_score.append(score)
                elif time >= 24 and time < 48:
                    score = reliability_score[-1] - 10
                    reliability_score.append(score)
                elif time >= 48 and time < 120:
                    score = reliability_score[-1] - 7
                    reliability_score.append(score)
                else:
                    score = reliability_score[-1] - 5
                    reliability_score.append(score)
            else:
                score = reliability_score[-1] + 2
                reliability_score.append(score)

In [115]:
# set a cap to the list 0 - 100
reliability_score = [max(min(x, 100), 0) for x in reliability_score]
df['reliability_score'] = reliability_score

### <font color = green>output feature dataframe

In [116]:
df.columns

Index(['id', 'user_id', 'shift_id', 'facility_id', 'withdrawninfo_value',
       'status', 'prevStatus', 'distance', 'facility_id', 'shift_description',
       'assigned_nurse_id', 'net_pay', 's_unit', 'type', 's_qualifications',
       's_breakTime', 'withdrawnInfo', 'f_name', 'f_short_name', 'segmentName',
       'areaName', 's_create', 'sa_create', 'u_approve', 'u_create',
       'sa_statusupdate', 'Start_Time', 'U_create2U_approve', 'U_create2now',
       'U_approve2now', 'S_create2SA_Create', 'SA_Create2Start_Time',
       'S_Create2Start_Time', 'CW_Time2Start_Time', 'target', 'f_highrate',
       'f_lowrate', 'count_prev_SA', 'count_prev_CW', 'prev_CW/SA_rate',
       'prev_CW x SA_rate', 'type_RN', 'type_LVN+LPN', 'type_STNA', 'type_CNA',
       'segmentName_d', 'areaName_houston', 'areaName_no', 'areaName_dfw',
       'areaName_austin', 'areaName_san', 'Str_Start_Time', 'start_date',
       'CW_in_a_month', 'reliability_score'],
      dtype='object')

In [117]:
feature_df = df[['id', 'user_id', 'shift_id', 'prev_CW/SA_rate', 'status', 'S_create2SA_Create',
                 'S_Create2Start_Time','SA_Create2Start_Time','U_create2now', 'U_approve2now','prev_CW x SA_rate',
                 'type_RN', 'type_LVN+LPN', 'type_STNA','segmentName_d', 'areaName_houston', 'areaName_no', 'areaName_dfw', 
                 'areaName_austin', 'areaName_san', 'net_pay', 'target', 'sa_create', 'Start_Time',
                 'CW_in_a_month', 'count_prev_SA', 'count_prev_CW','f_highrate','f_lowrate','type_CNA','reliability_score']]
feature_df.to_csv('model_data_new.csv')

In [118]:
# An summery of reliability score feature
df['reliability_score'].describe()

count    77063.000000
mean        56.763778
std         34.826829
min          0.000000
25%         31.000000
50%         58.000000
75%         94.000000
max        100.000000
Name: reliability_score, dtype: float64

In [119]:
feature_df.shape

(77063, 31)